In [19]:
import pandas as pd
from functools import reduce

In [20]:
# Read the 'NOTEEVENTS.csv'
note=pd.read_csv('NOTEEVENTS.csv')

/Users/gaoyueyi/opt/anaconda3/envs/cs/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
note.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [22]:
# Read the 'DIAGNOSES_ICD.csv'
icd=pd.read_csv('DIAGNOSES_ICD.csv')

In [23]:
icd.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [24]:
# Combine the 'DIAGNOSES_ICD.csv' and 'NOTEEVENTS.csv' based on 'ROW_ID'
note_icd=[note,icd]
note_icd_m=reduce(lambda left,right: pd.merge(left,right,on=['ROW_ID','ROW_ID']),note_icd)

In [25]:
note_icd_m.head()

,ROW_ID,SUBJECT_ID_x,HADM_ID_x,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,SUBJECT_ID_y,HADM_ID_y,SEQ_NUM,ICD9_CODE
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,23,152223,7.0,60000
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...,23,152223,8.0,3899
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...,24,161859,1.0,41041
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...,24,161859,2.0,41401
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...,24,161859,3.0,53081


In [26]:
nim = note_icd_m[['ROW_ID','TEXT','ICD9_CODE']]
nim

,ROW_ID,TEXT,ICD9_CODE
0,174,Admission Date: [**2151-7-16**] Dischar...,60000
1,175,Admission Date: [**2118-6-2**] Discharg...,3899
2,176,Admission Date: [**2119-5-4**] D...,41041
3,177,Admission Date: [**2124-7-21**] ...,41401
4,178,Admission Date: [**2162-3-3**] D...,53081
...,...,...,...
651042,649359,Chief Complaint: low back pain\n 24 Hour Eve...,3004
651043,649368,Chief Complaint: hemoptysis\n 24 Hour Events...,496
651044,649371,Chief Complaint: low back pain\n 24 Hour Eve...,V850
651045,649752,Chief Complaint: Unresponsiveness\n 24 Hour ...,99731


In [27]:
# Only retain information containing the chief complaint
note_icd_cc = pd.DataFrame(columns=['ROW_ID','TEXT','ICD9_CODE'])
count=0
for i in range(0,651047):
    if "Chief Complaint" in nim['TEXT'].iloc[i]:
        if "Chief Complaint: none" not in nim['TEXT'].iloc[i] and "Chief Complaint:\n   24 Hour" not in nim['TEXT'].iloc[i]:
            note_icd_cc=note_icd_cc.append({'ROW_ID': nim['ROW_ID'].iloc[i],
                                'TEXT': nim['TEXT'].iloc[i],
                                'ICD9_CODE' : nim['ICD9_CODE'].iloc[i]
                                },ignore_index=True)


In [28]:
note_icd_cc.to_csv('all_note_icd.csv')
note_icd_cc

,ROW_ID,TEXT,ICD9_CODE
0,176,Admission Date: [**2119-5-4**] D...,41041
1,177,Admission Date: [**2124-7-21**] ...,41401
2,178,Admission Date: [**2162-3-3**] D...,53081
3,179,Admission Date: [**2172-3-5**] D...,25000
4,180,Admission Date: [**2112-12-8**] ...,41071
...,...,...,...
89174,649359,Chief Complaint: low back pain\n 24 Hour Eve...,3004
89175,649368,Chief Complaint: hemoptysis\n 24 Hour Events...,496
89176,649371,Chief Complaint: low back pain\n 24 Hour Eve...,V850
89177,649752,Chief Complaint: Unresponsiveness\n 24 Hour ...,99731


In [33]:
# Roughly get the chief complaint information by observing the format of the text
cc_icd = pd.DataFrame(columns=['ROW_ID','TEXT','ICD9_CODE'])
count=0
for i in range(0,89179):
    temp=note_icd_cc['TEXT'].iloc[i]
    if "Chief Complaint:" in temp:
        temp=temp.split("Chief Complaint:",1)[1]
    if "Chief Complaint" in temp:
        temp=temp.split("Chief Complaint",1)[1]
    if "Major" in temp:
        temp = temp.split("Major",1)[0]
    if "24 Hour Events" in temp:
        temp = temp.split("24 Hour Events",1)[0]
    if "HPI" in temp:
        temp = temp.split("HPI",1)[0]
    if "History of" in temp:
        temp = temp.split("History of",1)[0]
    temp=temp.replace('\n','')
    temp=temp.replace(':','')
    if len(temp) < 2:
        continue
    cc_icd=cc_icd.append({'ROW_ID': note_icd_cc['ROW_ID'].iloc[i],
                        'TEXT': temp,
                        'ICD9_CODE' : note_icd_cc['ICD9_CODE'].iloc[i]
                        },ignore_index=True)
# cc_icd=cc_icd.dropna()
# cc_icd.to_csv('cc_icd.csv')

In [38]:
cc_icd=cc_icd.dropna()
cc_icd.to_csv('cc_icd.csv')
cc_icd

,ROW_ID,TEXT,ICD9_CODE
0,176,"81 yo F smoker w/ COPD, severe TBM, s/p trache...",41041
1,177,COPD exacerbation/Shortness of Breath,41401
2,178,Mr. [**Known lastname 1829**] was seen at [**H...,53081
3,179,Meningioma,25000
4,180,Hypoxia,41071
...,...,...,...
89141,649359,low back pain,3004
89142,649368,hemoptysis,496
89143,649371,low back pain,V850
89144,649752,Unresponsiveness,99731


In [40]:
# Three-Digit Category
cc_icd_3d = pd.DataFrame(columns=['ROW_ID','TEXT','ICD9_CODE'])
for i in range(0,len(cc_icd)):
    icd9_all = cc_icd['ICD9_CODE'].iloc[i]
    if icd9_all[0]== 'E' or icd9_all[0]== 'V':
        continue
    else:
        icd9_3d = icd9_all[:3]
    cc_icd_3d=cc_icd_3d.append({'ROW_ID': cc_icd['ROW_ID'].iloc[i],
                        'TEXT': cc_icd['TEXT'].iloc[i],
                        'ICD9_CODE' : icd9_3d
                        },ignore_index=True)
cc_icd_3d

,ROW_ID,TEXT,ICD9_CODE
0,176,"81 yo F smoker w/ COPD, severe TBM, s/p trache...",410
1,177,COPD exacerbation/Shortness of Breath,414
2,178,Mr. [**Known lastname 1829**] was seen at [**H...,530
3,179,Meningioma,250
4,180,Hypoxia,410
...,...,...,...
75936,649480,Diarrhea and weakness,518
75937,649359,low back pain,300
75938,649368,hemoptysis,496
75939,649752,Unresponsiveness,997


In [41]:
cc_icd_3d.to_csv('cc_icd_3d.csv')

In [42]:
# Chapter
cc_icd_chapter = pd.DataFrame(columns=['ROW_ID','TEXT','ICD9_CHAPTER'])
for i in range(0,len(cc_icd_3d)):
    icd9_3d = cc_icd_3d['ICD9_CODE'].iloc[i]
    icd9_chapter = ''
    if int(icd9_3d) <= 139:
        icd9_chapter = '001-139'
    elif int(icd9_3d) <= 239:
        icd9_chapter = '140-239'
    elif int(icd9_3d) <= 279:
        icd9_chapter = '240-279'
    elif int(icd9_3d) <= 289:
        icd9_chapter = '280-289'
    elif int(icd9_3d) <= 319:
        icd9_chapter = '290-319'
    elif int(icd9_3d) <= 389:
        icd9_chapter = '320-389'
    elif int(icd9_3d) <= 459:
        icd9_chapter = '390-459'
    elif int(icd9_3d) <= 519:
        icd9_chapter = '460-519'
    elif int(icd9_3d) <= 579:
        icd9_chapter = '520-579'
    elif int(icd9_3d) <= 629:
        icd9_chapter = '580-629'
    elif int(icd9_3d) <= 679:
        icd9_chapter = '630-679'
    elif int(icd9_3d) <= 709:
        icd9_chapter = '680-709'
    elif int(icd9_3d) <= 739:
        icd9_chapter = '710-739'
    elif int(icd9_3d) <= 759:
        icd9_chapter = '740-759'
    elif int(icd9_3d) <= 779:
        icd9_chapter = '760-779'
    elif int(icd9_3d) <= 799:
        icd9_chapter = '780-799'
    elif int(icd9_3d) <= 999:
        icd9_chapter = '800-999'
    cc_icd_chapter=cc_icd_chapter.append({'ROW_ID': cc_icd_3d['ROW_ID'].iloc[i],
                        'TEXT': cc_icd_3d['TEXT'].iloc[i],
                        'ICD9_CHAPTER' : icd9_chapter
                        },ignore_index=True)
cc_icd_chapter

,ROW_ID,TEXT,ICD9_CHAPTER
0,176,"81 yo F smoker w/ COPD, severe TBM, s/p trache...",390-459
1,177,COPD exacerbation/Shortness of Breath,390-459
2,178,Mr. [**Known lastname 1829**] was seen at [**H...,520-579
3,179,Meningioma,240-279
4,180,Hypoxia,390-459
...,...,...,...
75936,649480,Diarrhea and weakness,460-519
75937,649359,low back pain,290-319
75938,649368,hemoptysis,460-519
75939,649752,Unresponsiveness,800-999


In [43]:
cc_icd_chapter.to_csv('cc_icd_chapter.csv')